# 競馬データの収集
## 競馬データの収集元ネタ
スクレイピングの元ネタサイト。
https://zenn.dev/dijzpeb/books/848d4d8e47001193f3fb/viewer/604e12
## 取得データ


##0.ライブラリ情報

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from google.colab import drive
from csv import reader

##1.レース情報のスクレイピングを行う
取得範囲は、
　・YYYYPPKKDDRR
　　（開催年、開催場所、開催回、開催日、レース）
```
# race_id = "2019" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
```
※開催場所：
    札幌:01 函館:02 福島:03 新潟:04 東京:05 中山:06 中京:07 京都:08 阪神:09 小倉:10<br>
※開催回は、最大7回と仮定  
※開催日は、最大12日と仮定   
※開催レースは最大12Rを仮定  
(2021/6/21:2018,2019,2020情報取得済み。それより過去は要検討。2021年は未取得：新規追加データの継続的取得方法は要検討) 

###テストコード

In [ ]:
race_id = "201901010101"
url = "https://db.netkeiba.com/race/{0}".format(race_id)
pd.read_html(url)[0]

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
0,1,1,1,ゴルコンダ,牡2,54,ルメール,1:48.3,NaN,1.4,1,518(-16),[東] 木村哲也
1,2,3,3,プントファイヤー,牡2,54,岩田康誠,1:50.1,大,3.5,2,496(-8),[東] 手塚貴久
2,3,4,4,ラグリマスネグラス,牡2,51,団野大成,1:50.9,5,46.6,6,546(+6),[東] 藤沢和雄
3,4,8,9,キタノコドウ,牡2,51,菅原明良,1:51.5,3.1/2,56.8,7,458(-8),[東] 高木登
4,5,5,5,ネモフィラブルー,牡2,54,川島信二,1:51.7,1.1/2,140.3,9,436(0),[西] 矢作芳人
5,6,8,8,マイネルラクスマン,牡2,54,丹内祐次,1:52.1,2.1/2,9.7,3,480(+8),[東] 金成貴史
6,7,2,2,サンモンテベロ,牝2,54,黛弘人,1:52.5,2.1/2,114.7,8,450(+2),[東] 中野栄治
7,8,7,7,エスカレーション,牝2,54,藤岡佑介,1:52.5,アタマ,26.1,5,448(-4),[東] 高柳瑞樹
8,9,6,6,セイウンジュリア,牝2,54,池添謙一,1:52.6,クビ,16.4,4,470(0),[西] 浅見秀一


###取得したレースデータをcsv化する

In [ ]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """

        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
          time.sleep(1)
          try:
              url = "https://db.netkeiba.com/race/" + race_id
              #メインとなるテーブルデータを取得
              df = pd.read_html(url)[0]

              html = requests.get(url)
              html.encoding = "EUC-JP"
              soup = BeautifulSoup(html.text, "html.parser")

              #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
              texts = (
                  soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                  + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
              )
              info = re.findall(r'\w+', texts)
              for text in info:
                  if text in ["芝", "ダート"]:
                      df["race_type"] = [text] * len(df)
                  if "障" in text:
                      df["race_type"] = ["障害"] * len(df)
                  if "m" in text:
                      df["course_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                  if text in ["良", "稍重", "重", "不良"]:
                      df["ground_state"] = [text] * len(df)
                  if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                      df["weather"] = [text] * len(df)
                  if "年" in text:
                      df["date"] = [text] * len(df)

              #馬ID、騎手IDをスクレイピング
              horse_id_list = []
              horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                  "a", attrs={"href": re.compile("^/horse")}
              )
              for a in horse_a_list:
                  horse_id = re.findall(r"\d+", a["href"])
                  horse_id_list.append(horse_id[0])
              jockey_id_list = []
              jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                  "a", attrs={"href": re.compile("^/jockey")}
              )
              for a in jockey_a_list:
                  jockey_id = re.findall(r"\d+", a["href"])
                  jockey_id_list.append(jockey_id[0])
              df["horse_id"] = horse_id_list
              df["jockey_id"] = jockey_id_list

              #インデックスをrace_idにする
              df.index = [race_id] * len(df)

              race_results[race_id] = df
           #存在しないrace_idを飛ばす
          except IndexError:
              continue
          #wifiの接続が切れた時などでも途中までのデータを返せるようにする
          except Exception as e:
              print(e)
              break
          #Jupyterで停止ボタンを押した時の対処
          except:
              break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df

race_id_list = []
for place in range(6, 11, 1):
    for kai in range(1, 8, 1):
        for day in range(1, 13, 1):
            for r in range(1, 13, 1):
                race_id = "2018" + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                race_id_list.append(race_id)

results = Results.scrape(race_id_list)

path="/content/drive/My Drive/keiba/"
results.to_csv(path + "racedata_2018_06-10.csv")

##2.競走馬の過去成績データのスクレイピングを行う
(2021/6/21)レース情報をある程度収集したのち、その中の馬IDを集約してからこっちの収集を行う。  
目的：レース情報より詳細なデータ。簡易な血統データの取得。

###テストコード

In [ ]:
horse_id = "2017104140"
url = "https://db.netkeiba.com/horse/{0}".format(horse_id)
print(pd.read_html(url)[1])
print(pd.read_html(url)[2])
print(pd.read_html(url)[3])

###競走馬IDの抽出
レースデータから競走馬IDを抽出する。  
レース情報で取得した馬IDのみを取り出して、集約したcsvを作成する。

In [ ]:
path="/content/drive/My Drive/keiba/"
df_firsthalf = pd.read_csv(path + "racedata_2020_01-05.csv")
df_latterhalf = pd.read_csv(path + "racedata_2020_06-10.csv")

#それぞれのcsvデータからhorse_idをseriesで抽出し、結合・distinctする。
sr_horse_id_first = df_firsthalf['horse_id']
sr_horse_id_latter = df_latterhalf['horse_id']
sr_horse_id = pd.concat([sr_horse_id_first,sr_horse_id_latter]).drop_duplicates()

#dfでやると、行列データとして扱われてしまうため、最後のtolist時にlist in list構造となってしまうため、
#一貫してseriesデータで加工を行う。
#df_horse_id_first = df_firsthalf.loc[:,['horse_id']]
#df_horse_id_latter = df_latterhalf.loc[:,['horse_id']]
#df_horse_id = pd.concat([df_horse_id_first,df_horse_id_latter])
#print(df_horse_id.drop_duplicates())

#抽出したhorse_id seriesデータを、リストに変換する。
#horse_id_list = sr_horse_id.values.tolist()
#print(horse_id_list)

#リストの要素数が多く、スクレイピング処理が終わらないため、馬IDを四等分する。
#def chunkIt(seq, num):
#    avg = len(seq) / float(num)
#    out = []
#    last = 0.0

#    while last < len(seq):
#        out.append(seq[int(last):int(last + avg)])
#        last += avg

#    return out

#print(len(horse_id_list))
#list = chunkIt(horse_id_list, 4)
#print(len(list[0]))
#print(len(list[1]))
#print(len(list[2]))
#print(len(list[3]))

#もっと上手く書きたい...
#horse_id_list_1 = pd.Series(list[0])
#horse_id_list_2 = pd.Series(list[1])
#horse_id_list_3 = pd.Series(list[2])
#horse_id_list_4 = pd.Series(list[3])

path="/content/drive/My Drive/keiba/"
sr_horse_id.to_csv(path + "horse_id_list_2020.csv")
print(sr_horse_id)
#horse_id_list_1.to_csv(path + "horse_id_list_2018_01.csv")
#horse_id_list_2.to_csv(path + "horse_id_list_2018_02.csv")
#horse_id_list_3.to_csv(path + "horse_id_list_2018_03.csv")
#horse_id_list_4.to_csv(path + "horse_id_list_2018_04.csv")


###過去成績のデータを競走馬IDを元に取得

In [ ]:
#馬の過去成績データを処理するクラス
class HorseResults:
    @staticmethod
    def scrape(horse_id_list):
        """
        馬の過去成績データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        horse_results_df : pandas.DataFrame
            全馬の過去成績データをまとめてDataFrame型にしたもの
        """

        #horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = 'https://db.netkeiba.com/horse/' + str(horse_id)
                df = pd.read_html(url)[3]
                #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0]=='受賞歴':
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
                print(horse_results)
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
        
        #pd.DataFrame型にして一つのデータにまとめる        
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        return horse_results_df


path="/content/drive/My Drive/Colab Notebooks/"
df = pd.read_csv(path + "horse_id_list_2019_01.csv")
print(df['0'])

#抽出したhorse_id seriesデータを、リストに変換する。
horse_id_list = df['0'].tolist()
results = HorseResults.scrape(horse_id_list)
results.to_csv(path + "horsedata_2019_01.csv")

##3.競走馬の血統データのスクレイピングを行う
(2021/6/21)1.のレースデータを取得した後、2.の馬情報を取ってくる。その後に、馬IDを元に取ってくるようにする。  
ただ、2.の馬情報で祖父母までは取れるので、それでいい気もしている。

In [ ]:
#血統データを処理するクラス
class Peds:
    @staticmethod
    def scrape(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """

        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = "https://db.netkeiba.com/horse/ped/" + horse_id
                df = pd.read_html(url)[0]

                #重複を削除して1列のSeries型データに直す
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([i], axis=1, inplace=True)
                    df = df.drop_duplicates()
                ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)

                peds_dict[horse_id] = ped.reset_index(drop=True)
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #列名をpeds_0, ..., peds_61にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')

        return peds_df

##4.払い戻し情報のスクレイピングを行う
回収率計算のため。  
(2021/6/21)まだやんなくていいと思ってる。

In [ ]:
#払い戻し表データを処理するクラス
class Return:
    @staticmethod
    def scrape(race_id_list):
        """
        払い戻し表データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        return_tables_df : pandas.DataFrame
            全払い戻し表データをまとめてDataFrame型にしたもの
        """

        return_tables = {}
        for race_id in tqdm(race_id_list):
            try:
                url = "https://db.netkeiba.com/race/" + race_id

                #普通にスクレイピングすると複勝やワイドなどが区切られないで繋がってしまう。
                #そのため、改行コードを文字列brに変換して後でsplitする
                f = urlopen(url)
                html = f.read()
                html = html.replace(b'<br />', b'br')
                dfs = pd.read_html(html)

                #dfsの1番目に単勝〜馬連、2番目にワイド〜三連単がある
                df = pd.concat([dfs[1], dfs[2]])

                df.index = [race_id] * len(df)
                return_tables[race_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break

        #pd.DataFrame型にして一つのデータにまとめる
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        return return_tables_df

# 将来的なメモ
## データの整備<br>
　これが一番めんどくさそう・・・
　テーブルデータの作成⇨データクレンジング・レースデータの馬IDに血統データを紐つける  
1. まずは都市伝説の実証をやってみる  
 + 父系/母父系の血統で見てみる⇨父系・母父系の馬に絞ってIDを付けてみる
 + 体重の傾向で見てみる⇨年齢・前回のレースからの期間
 + 米血統/欧州血統/日本血統で分類してみる⇨主成分分析？
 + （あとは調べて見てみる）  
1. データを機械学習モデルにぶっ込んでみる⇨どんな機械学習モデルが存在するか？  あとは交差検証
1.・・・ここから先はとりあえずやって見て考える

#雑多なコード

In [ ]:
#データの重複を除く（Unnamed: 0は任意の列名に変える）
from google.colab import drive
path="/content/drive/My Drive/Colab Notebooks/"
df = pd.read_csv(path + "racedata_2020_01-05.csv")
print(df['Unnamed: 0'].drop_duplicates())

#馬IDの取得
